# Merge data sets into training and validation sets without droping signals

In [28]:
import pandas as pd
import numpy as np

In [29]:
# load the training data sets
landsat_df = pd.read_csv('Benchmark/landsat_features_training.csv')
terraclimate_df = pd.read_csv('Benchmark/terraclimate_features_training.csv')
wq_df = pd.read_csv('Benchmark/water_quality_training_dataset.csv')
landsat_df.shape, terraclimate_df.shape, wq_df.shape

((9319, 9), (9319, 4), (9319, 6))

In [30]:
# Combine two datasets vertically (along columns) using pandas concat function.
def combine_datasets(dataset1,dataset2,dataset3):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    
    data = pd.concat([dataset1,dataset2,dataset3], axis=1)
    data = data.loc[:, ~data.columns.duplicated()]
    return data

In [31]:
df = combine_datasets(landsat_df, terraclimate_df, wq_df)
df.head()

,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI,pet,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,02-01-2011,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595,174.2,128.912,555.0,10.0
1,-26.861111,28.884722,03-01-2011,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,124.1,74.720,162.9,163.0
2,-26.450000,28.085833,03-01-2011,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,127.5,89.254,573.0,80.0
3,-27.671111,27.236944,03-01-2011,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,129.7,82.000,203.6,101.0
4,-27.356667,27.286389,03-01-2011,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,129.2,56.100,145.1,151.0


In [32]:
df.shape, df.columns, df.info(), df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9319 entries, 0 to 9318
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Latitude                       9319 non-null   float64
 1   Longitude                      9319 non-null   float64
 2   Sample Date                    9319 non-null   object 
 3   nir                            8234 non-null   float64
 4   green                          8234 non-null   float64
 5   swir16                         8234 non-null   float64
 6   swir22                         8234 non-null   float64
 7   NDMI                           8234 non-null   float64
 8   MNDWI                          8234 non-null   float64
 9   pet                            9319 non-null   float64
 10  Total Alkalinity               9319 non-null   float64
 11  Electrical Conductance         9319 non-null   float64
 12  Dissolved Reactive Phosphorus  9319 non-null   f

((9319, 13),
 Index(['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16',
        'swir22', 'NDMI', 'MNDWI', 'pet', 'Total Alkalinity',
        'Electrical Conductance', 'Dissolved Reactive Phosphorus'],
       dtype='object'),
 None,
           Latitude    Longitude           nir         green        swir16  \
 count  9319.000000  9319.000000   8234.000000   8234.000000   8234.000000   
 mean    -28.474988    26.868414  14045.485426   9983.213141  13567.459315   
 std       2.760282     3.535164   2953.223626   2778.780177   3348.517657   
 min     -34.405833    17.730278   3992.000000   4045.000000   3672.500000   
 25%     -30.160091    26.126667  12723.625000   9370.000000  11760.625000   
 50%     -28.058889    27.409060  14183.000000   9801.000000  13704.250000   
 75%     -26.861111    29.245556  15513.875000  10286.000000  15425.625000   
 max     -22.225556    32.325000  65535.000000  65535.000000  65535.000000   
 
              swir22         NDMI        MNDWI  

In [33]:
df.isna().sum()

Latitude                            0
Longitude                           0
Sample Date                         0
nir                              1085
green                            1085
swir16                           1085
swir22                           1085
NDMI                             1085
MNDWI                            1085
pet                                 0
Total Alkalinity                    0
Electrical Conductance              0
Dissolved Reactive Phosphorus       0
dtype: int64

In [34]:
# Merging the validation data sets
# load the validation data sets
landsat_val_df = pd.read_csv('Benchmark/landsat_features_validation.csv')
terraclimate_val_df = pd.read_csv('Benchmark/terraclimate_features_validation.csv')
landsat_val_df.head(), terraclimate_val_df.head()

(    Latitude  Longitude Sample Date      nir    green   swir16   swir22  \
 0 -32.043333  27.822778  01-09-2014  15229.0  12868.0  14797.0  12421.0   
 1 -33.329167  26.077500  16-09-2015      NaN      NaN      NaN      NaN   
 2 -32.991639  27.640028  07-05-2015  16221.0   9304.5  12536.5   9958.0   
 3 -34.096389  24.439167  07-02-2012      NaN      NaN      NaN      NaN   
 4 -32.000556  28.581667  01-10-2014   9125.0  11100.5   9455.0   8711.0   
 
        NDMI     MNDWI  
 0  0.014388 -0.069727  
 1       NaN       NaN  
 2  0.128123 -0.147979  
 3       NaN       NaN  
 4 -0.017761  0.080052  ,
     Latitude  Longitude Sample Date        pet
 0 -32.043333  27.822778  01-09-2014  161.90001
 1 -33.329167  26.077500  16-09-2015  177.60000
 2 -32.991639  27.640028  07-05-2015  158.40001
 3 -34.096389  24.439167  07-02-2012  130.00000
 4 -32.000556  28.581667  01-10-2014  152.50000)

In [35]:
landsat_val_df.shape, terraclimate_val_df.shape

((200, 9), (200, 4))

In [36]:
validation_df = pd.concat([landsat_val_df, terraclimate_val_df], axis=1)
validation_df = validation_df.loc[:, ~validation_df.columns.duplicated()]
validation_df.head()

,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI,pet
0,-32.043333,27.822778,01-09-2014,15229.0,12868.0,14797.0,12421.0,0.014388,-0.069727,161.90001
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN,NaN,NaN,NaN,177.60000
2,-32.991639,27.640028,07-05-2015,16221.0,9304.5,12536.5,9958.0,0.128123,-0.147979,158.40001
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN,NaN,NaN,NaN,130.00000
4,-32.000556,28.581667,01-10-2014,9125.0,11100.5,9455.0,8711.0,-0.017761,0.080052,152.50000


In [37]:
df.to_csv('Training_Dataset.csv', index = False)
validation_df.to_csv('Validation_Dataset.csv', index = False)